In [ ]:
import pandas

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')

In [ ]:
os.environ.get('OPENAI_API_KEY')

In [ ]:
serper_api_key = os.environ.get('SERPER_API_KEY')

In [ ]:
file_path = "C:\\Users\\MGroup\\Downloads\\eyfdr_oscrm_contractor_x_articul.json"

In [ ]:
import json
import pandas as pd
import re
import requests

In [ ]:
with open(file_path, 'r', encoding='utf-8') as f:
    data = f.read()

In [ ]:
def escape_json_value(json_data):
    # Escape the necessary characters
    replacements = {
        '\\': '\\\\',  # Backslash
        '"': '\\"',    # Quotation mark
        '/': '\\/',    # Forward slash
        '\b': '\\b',   # Backspace
        '\f': '\\f',   # Formfeed
        '\n': '\\n',   # Newline
        '\r': '\\r',   # Carriage return
        '\t': '\\t'    # Horizontal tab
    }
    # print(json_data)
    for key, value in replacements.items():
        json_data = json_data.replace(key, value)

    return json_data


def escape_json_values(raw_data):
    # pattern = r'\"([^\"]+)\":\"([^\"]*)\"'

    pattern = r'\"([^\"]+)\":\"((?:\\.|[^\\"])*)\"'

    def replacement(match):
        # Escape special characters in the value part
        escaped_value = escape_json_value(match.group(2))
        return f'"{match.group(1)}":"{escaped_value}"'

    # Replace each match in the raw data
    return re.sub(pattern, replacement, raw_data)


In [ ]:
escaped_json = escape_json_values(data)


In [ ]:
data = json.loads(escaped_json)

df = pd.DataFrame(data[2]['data'])

print("Missing values in each column:\n", df.isnull().sum())

print("\nFirst few rows of the DataFrame:")
df.head()

In [ ]:
mask = df['description'].str.contains('E\+H', case=False, na=False)


In [ ]:
filtered_df = df[mask]
filtered_df.head()

In [ ]:
filtered_df

In [ ]:
markdown_table = filtered_df.to_markdown(index=False)

print(markdown_table)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Example DataFrame

filtered_df = df[df['title'] == 'Endress+Hauser']

# Join all descriptions into a single text
text = " ".join(df['description'].tolist())

# Extract bi-grams or tri-grams
vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words='english')
ngrams = vectorizer.fit_transform([text])
ngram_features = vectorizer.get_feature_names_out()

# Display most common n-grams
ngram_counts = ngrams.toarray().sum(axis=0)
ngram_df = pd.DataFrame({'ngram': ngram_features, 'count': ngram_counts})
ngram_df = ngram_df.sort_values(by='count', ascending=False)
print(ngram_df.head(20))  

In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg") 

In [ ]:
for description in filtered_df['description'].tolist():
    doc = nlp(description)

    for ent in doc.ents:
        if ent.label_ == "ORG":
            print(ent.text)

In [ ]:
for ent in doc.ents:
    if ent.label_ == "ORG":
        print(ent.text)

In [ ]:
raw_email = """

Buenas tardes,

Por medio del presente, solicito cotizar los siguientes productos de acuerdo con siguiente información:

                 Descripción

Marca

Numero de Parte   

Cantidad

INTERRUPTOR FLOTADOR 250VAC   150VDC 3A (AC) 1A (DC) (MAXIMO); DIMENSIONES: LARGO CABLE 20m

ENDRESS+HAUSER

71035522

7

 

                                                       

Por favor enviar ficha técnica y el mejor precio y plazo de entrega que nos pueden entregar.

 

A continuación, los datos para la cotización:

Soluciones Eléctricas SELEC SPA

Rut: 76917494-K 

Dirección: Av. Pedro de valdivia 273 Oficina 1411, Providencia Santiago Chile

Giro: Venta al por menor por correo, internet y vía telefónica.

 

Muchas Gracias

"""

In [ ]:
doc = nlp(raw_email)

for ent in doc.ents:
    if ent.label_ == "ORG":
        print(ent.text)

In [ ]:
import json

In [ ]:
def search(query):
    url = "https://google.serper.dev/search"
    
    payload = json.dumps({
      "q": query,
      "gl": "us",
      "page": 1
    })

    headers = {
        'X-API-KEY': serper_api_key,
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)

    return response.text

In [ ]:
gresult = json.loads(search('E+E EE741'))

In [ ]:
gresult = json.loads(gresult)

In [ ]:
for item in gresult['organic']:
    print(f'{item["title"]}\n{item["link"]}\n{item["snippet"]}\n\n')